In [20]:
import numpy as np
import time
import pandas as pd
from scipy.stats import qmc, norm

In [10]:
S0 = 100
K = 100
r = 0.05
sigma = 0.1

In [4]:
T = 252
N = 1
dt = N / T
sqrt_dt = np.sqrt(dt)

In [166]:
def payoff_european(S, call = True):
    if call:
        return np.maximum(S[-1] - K, 0)
    else:
        return np.maximum(K - S[-1], 0)

def payoff_asian(S, call = True):
    if call:
        return np.maximum(S.mean(axis = 0) - K, 0)
    else:
        return np.maximum(K - S.mean(axis = 0), 0)
    
def payoff_asian_importance(S, likelihood_ratio = 0.5, call=True):
    raw_payoff = np.maximum(S.mean(axis=0) - K, 0) if call else np.maximum(K - S.mean(axis=0), 0)
    return raw_payoff * likelihood_ratio

In [41]:
sampler = qmc.Sobol(d=1, scramble=True)
u = sampler.random(10)
u.squeeze()

C:\Users\starr\AppData\Local\Temp\ipykernel_19232\1749245857.py:2: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  u = sampler.random(10)


array([0.51634644, 0.30371271, 0.02498414, 0.81069983, 0.89326006,
       0.17681757, 0.40195722, 0.68374547, 0.72182543, 0.44199892])

In [70]:
def quasi_random_normal_samples(n, dim = 1):
    sampler = qmc.Sobol(d=dim, scramble=True)
    u = sampler.random(n)
    u = u.squeeze()
    z = norm.ppf(u)

    return z

In [165]:
sampler = qmc.Sobol(d=1, scramble=True)
u = sampler.random(256)
u = u.squeeze()
z = norm.ppf(u)
z.mean(), z.std()

C:\Users\starr\anaconda3\lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


(-inf, nan)

In [67]:
def quasi_brownian_asset(S_i, n):
    Z = quasi_random_normal_samples(n)
    return S_i * (1 + r * dt + sigma * sqrt_dt * Z)

def brownian_asset(S_i, n):
    Z = np.random.normal(size = n)    
    return S_i * (1 + r * dt + sigma * sqrt_dt * Z)

def antithetic_brownian_asset(S_i, n):
    Z1 = np.random.normal(size = (int) (n/2))
    Z2 = -Z1
    
    Z = np.empty((Z1.size + Z2.size,), dtype=Z1.dtype)
    Z[0::2] = Z1
    Z[1::2] = Z2
    
    return S_i * (1 + r * dt + sigma * sqrt_dt * Z)

def brownian_asset_importance_sampling(S_i, n, mu_shift=0.2):
    Z = np.random.normal(loc=mu_shift, size=n)
    S_path = S_i * (1 + r * dt + sigma * sqrt_dt * Z)
    
    # Likelihood ratio for each Z sample
    likelihood_ratio = np.exp(-mu_shift * Z + 0.5 * mu_shift**2)
    
    return S_path, likelihood_ratio

In [15]:
def run_monte_carlo(n, price, payoff):
    S = np.zeros((T, n)) + S0
    
    for i in range(1, T):
        S[i] = price(S[i-1], n)
        
    return payoff(S).mean(), payoff(S).std()

In [126]:
def run_experiment(method, payoff = payoff_asian):
    n_list = np.arange(1, 7, 1)
    v = np.zeros_like(n_list, dtype = np.float64)
    std_devs = np.zeros_like(n_list, dtype = np.float64)
    times = np.zeros_like(n_list, dtype = np.float64)

    for i in range(len(n_list)):
        start_time = time.time()
        n = n_list[i]
        v[i], std_devs[i] = run_monte_carlo(2 ** n, method, payoff)
        times[i] = time.time() - start_time
        
    return (v, std_devs, times)

In [167]:
run_experiment(brownian_asset, payoff = payoff_european)

(array([14.63418577, 12.13763764,  8.40495644,  7.73993668,  8.57141584,
         8.05371281]),
 array([ 2.13411169,  7.42429652, 11.36073177,  8.35379261,  8.39247029,
         7.90960245]),
 array([0.00282216, 0.0037806 , 0.0015161 , 0.00407529, 0.        ,
        0.00812459]))

In [168]:
run_experiment(antithetic_brownian_asset, payoff_european)

(array([18.92675627,  5.63325075,  5.37057691,  8.90214009,  5.83261614,
         7.15840452]),
 array([18.92675627,  4.88397983,  4.78558269,  9.95545415,  6.18270645,
         7.7889781 ]),
 array([0.005651  , 0.00448203, 0.00093031, 0.00431585, 0.00392485,
        0.00288391]))

In [144]:
run_experiment(antithetic_brownian_asset, payoff_asian_importance)

(array([2.02851559, 1.96871753, 1.93455062, 1.88055961, 1.90737117,
        1.90493511]),
 array([3.14354897e-03, 3.35454941e-03, 6.23559952e-03, 5.36286831e-02,
        8.87056112e-01, 1.04932346e+01]))

In [169]:
run_experiment(quasi_brownian_asset, payoff_european)

(array([7.05547244, 5.01807274, 6.44029423, 5.39100577, 5.95940337,
        5.17600188]),
 array([6.97118165, 3.15065988, 3.9744513 , 5.17847153, 6.38071304,
        4.15256337]),
 array([0.23583412, 0.306216  , 0.32207894, 0.33076835, 0.31311178,
        0.31271029]))